In [6]:
import pandas as pd
import numpy as np


In [32]:
dfPath = "/home/alextay96/Desktop/workspace/dmg_price/data/complete_encoded.csv"
df = pd.read_csv(dfPath)
df.head()

,Unnamed: 0.1,Unnamed: 0,Model,Make,CaseID,Vehicle_Model,DamageLoss_Type,Reason_Flag,NCB_Stat,Car_Age,...,enc_Vehicle_Type,enc_Claim_Type,enc_Collision_With,enc_Loss_State,enc_Third_Party_Involved,enc_Vehicle_Still_Driveable,enc_DamageLoss_Type,enc_Initial_Estimation,enc_Handling_Insurer,enc_Condition_Of_Damage
0,0,0,Ranger 2015 On,FORD,10198027,FORD Ranger 2015 On Dr Pickup Dbl.Cab 2.2 TD X...,Collision,Payment Settled,0,3.0,...,41,4,9,17,1,1,0,13,23,0
1,1,1,Axia 2014 On,PERODUA,10198029,PERODUA Axia 2014 On 5 Dr Hatch 1.0 E 67Bhp (M...,Collision,Payment Settled,0,0.0,...,26,4,9,11,1,0,0,0,27,0
2,2,2,City 2014 On,HONDA,10198144,HONDA City 2014 On 4 Dr Sal 1.5 V AUTO 118Bhp ...,Collision,Payment Settled,1,5.0,...,47,1,6,12,1,1,0,0,3,0
3,3,3,Wira 1993 On,PROTON,10198362,PROTON Wira 1993 On 4 Dr Sal 1.5 GL 86Bhp (M) ...,Collision,Payment Settled,1,10.0,...,47,6,9,13,0,2,0,22,8,0
4,4,4,Vios 2007 On,TOYOTA,10198035,TOYOTA Vios 2007 On 4 Dr Sal 1.5 G (NCP93R) AU...,Collision,Payment Settled,0,12.0,...,47,4,9,16,1,1,0,17,13,0


In [42]:
df2 = df.groupby(["Vehicle_Type", "Model"])["CaseID"].count().sort_values(ascending=False).iloc[:100]
df2.head(100)


Vehicle_Type           Model            
Hatchback - 5 Dr       Myvi 2011 On         31679
                       Axia 2014 To 2019    26118
MPV                    Alza 2009 On         22458
Saloon - 4 Dr          City 2014 On         21979
Hatchback - 5 Dr       Myvi 2005 On         20965
                                            ...  
SUV - 5 Dr             X50 2020 On            934
Pickup - 4 Dr Dbl.Cab  Navara 2015 On         906
Saloon - 4 Dr          E-Class 2009 On        898
MPV                    Serena 2011 On         889
SUV - 5 Dr             CX-5 2017 On           888
Name: CaseID, Length: 100, dtype: int64

In [44]:
df["model_name"] = df["Model"].apply(lambda x : x.split(" ")[0])
df2 = df.groupby(["Vehicle_Type", "model_name"])["CaseID"].count().sort_values(ascending=False)

df2.head(50)


Vehicle_Type           model_name
Hatchback - 5 Dr       Myvi          75889
Saloon - 4 Dr          Saga          38828
Hatchback - 5 Dr       Axia          33971
Saloon - 4 Dr          Vios          33804
                       City          31909
                       Bezza         27626
MPV                    Alza          22458
Saloon - 4 Dr          Persona       18147
Pickup - 4 Dr Dbl.Cab  Hilux         15435
Hatchback - 5 Dr       Viva          14154
Saloon - 4 Dr          Civic         11405
                       Almera        10594
Hatchback - 5 Dr       Jazz           8299
SUV - 5 Dr             HR-V           8143
MPV                    Avanza         7114
Saloon - 4 Dr          Camry          6295
SUV - 5 Dr             CR-V           6149
Saloon - 4 Dr          Corolla        6123
                       Waja           5687
MPV                    Exora          5665
Saloon - 4 Dr          Accord         5425
                       Wira           5133
MPV                 

In [43]:
import glob
srcDir = "/home/alextay96/Desktop/workspace/mrm_workspace/dmg_consistent_detection/data/filesmetadata"
allPath = glob.glob(f"{srcDir}/**/*.csv", recursive=True)
allDf = []
for i in allPath:
    allDf.append(pd.read_csv(i))
dd = pd.concat(allDf)
dd.to_csv("./selected.csv")

KeyboardInterrupt: 

In [49]:
df = pd.read_csv("/home/alextay96/Desktop/workspace/mrm_workspace/dmg_consistent_detection/data/filesmetadata/selected.csv", low_memory=False)
# df.dropna(inplace=True)
df2 = pd.read_csv( "/home/alextay96/Desktop/workspace/dmg_price/data/complete_encoded.csv")
df3 = pd.read_csv("/home/alextay96/Desktop/workspace/dmg_price/download_task.csv")
df = df.merge(df2, on="CaseID")
df = df.merge(df3[["CaseID","cost_pg_front", "cost_pg_rear"]], on="CaseID")

df["CaseID"] = df["CaseID"].astype(np.int64)
df.columns
vt = ["Saloon - 4 Dr", "Hatchback - 5 Dr", "SUV - 5 Dr", "MPV", "Pickup - 4 Dr Dbl.Cab"]
df = df[df["Vehicle_Type"].isin(vt)]
print(df.groupby("Vehicle_Type")["CaseID"].count().sort_values(ascending=False))
df["FinalDate"] = pd.to_datetime(df["FinalDate"])
df = df.sort_values("FinalDate", ascending=True).groupby(["CaseID", "StdDocDesc"]).head(1)
df = df.groupby(["Model", "cost_pg_front", "cost_pg_rear"]).head(20)

df = df.sort_values(["cost_pg_front", "cost_pg_rear"], ascending=False)
df = df[["Vehicle_Type", "Model", "StdDocDesc", "CaseID", "FinalDate", "cost_pg_front", "cost_pg_rear", "iDOCID"]]
print(df.head(50))
df.to_csv("./tasks.csv")



Vehicle_Type
Saloon - 4 Dr            67974
SUV - 5 Dr               27587
Hatchback - 5 Dr         26888
MPV                      20409
Pickup - 4 Dr Dbl.Cab    14474
Name: CaseID, dtype: int64
                Vehicle_Type                 Model  StdDocDesc    CaseID  \
289         Hatchback - 5 Dr          Myvi 2017 On  Front View  10199813   
290         Hatchback - 5 Dr          Myvi 2017 On   Rear View  10199813   
3823                     MPV        Innova 2005 On  Front View  10217340   
7404        Hatchback - 5 Dr         Swift 2005 On  Front View  10231456   
7405        Hatchback - 5 Dr         Swift 2005 On   Rear View  10231456   
12430                    MPV        Avanza 2004 On  Front View  10252404   
12431                    MPV        Avanza 2004 On   Rear View  10252404   
13900       Hatchback - 5 Dr          Myvi 2017 On   Rear View  10258253   
14607          Saloon - 4 Dr          Vios 2007 On  Front View  10261591   
14608          Saloon - 4 Dr          Vios 20

In [63]:
df = pd.read_csv( "/home/alextay96/Desktop/workspace/dmg_price/data/complete_encoded.csv")
df = df[df["Model"].str.contains("City")]
stdView = df.groupby(["Model"], as_index=False)["Sum_Insured"].std().rename(columns={'Sum_Insured':'std'})
meanView = df.groupby(["Model"], as_index=False)["Sum_Insured"].mean().rename(columns={'Sum_Insured':'mean'})
df2 = pd.concat([meanView, stdView],axis=1)

df2.head(30)


,Model,Sum_Insured,Model,Sum_Insured
0,City 1996 On,12595.454545,City 1996 On,14751.932150
1,City 1996 To 1997,15357.142857,City 1996 To 1997,18504.083303
2,City 1999 To 2003,11568.965517,City 1999 To 2003,8629.025008
3,City 2003 To 2005,14452.334559,City 2003 To 2005,2760.251455
4,City 2004 To 2005,17053.563218,City 2004 To 2005,7480.619983
5,City 2005 On,19087.647300,City 2005 On,4618.337565
6,City 2008 On,32106.746350,City 2008 On,7036.513716
7,City 2014 On,56773.362673,City 2014 On,15211.968174
8,City 2020 On,79062.621359,City 2020 On,10661.065121
